## Imports

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from joblib import dump, load
import warnings
warnings.filterwarnings('ignore') 


import librosa
from librosa import display

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## Load Data

In [2]:
all_genres = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
num_songs = 100
sr = 22050
Y_LIMIT = 660000

In [ ]:
class DataLoaderFMA():
    def __init__(self):
        self.X_mfcc = None
        self.Y = None
        
    def save_mfcc(self, genres=all_genres, songs=num_songs):
        assert(self.X_mfcc is None and self.Y is None)
        X_mfcc, Y = None, None
        for g_idx, g in enumerate(genres):
            for s_idx in range(songs):
                y, sr = librosa.load(f'genres/{g}/{g}.000{s_idx:02d}.wav')
                y = y[:Y_LIMIT]
                
                mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512, n_mfcc=13)
                
                if X_mfcc is None:
                    X_mfcc = mfcc.reshape(1, mfcc.shape[0])
                    Y = np.array([[g_idx]])
                else:
                    X_mfcc = np.vstack([X_mfcc, mfcc])
                    Y = np.vstack([Y, np.array([[g_idx]])])    
        Y = Y.ravel()
        
        self.X_mfcc = X_mfcc
        self.Y = Y
        
        np.savetxt('fma/Y.csv')
        np.savetxt('fma/X_mfcc.csv', X_mfcc)
        
    def load_mfcc(self):
        self.X_mfcc = np.loadtxt('data/X_mfcc.csv')
        
    def load_Y(self):
        self.Y = np.loadtxt('data/Y.csv')